In [1]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
from scipy import signal
import random
from scipy.optimize import curve_fit

import h5py
from IPython.utils import io

from joblib import Parallel, delayed
from tqdm import tqdm
import pandas as pd

In [2]:
%matplotlib widget

In [3]:
%reload_ext autoreload
%autoreload 2

In [4]:
import sys
sys.path.append('../../../../nadavp/new_trap/Tools/')
sys.path.append('../../../../nadavp/new_trap/Tools/StatFramework/')
import BeadDataFile
from discharge_tools import *

from likelihood_calculator import likelihood_analyser
from likelihood_calculator import GravityFramework
from likelihood_calculator import auxiliary_functions as aux
gfw = GravityFramework.GravityFramework()

sys.path.append('/home/analysis_user/New_trap_code/SensitivityFramework/')

### scale

In [11]:
# 5um
bdf_discharge = load_dir(dirname='/data/new_trap/20201116/Bead2/Discharge/', 
                         start_file=0, max_file=175)

2674  files in folder
175  files loaded


In [198]:
bandwidth = 2
m2 = gfw.build_z_response(bdf_discharge, drive_freq=71, charges=1, bandwidth=bandwidth, bimodal=False)

***************************************************
Z2-amplitude:  1.00e+04
reduced chi2:  7206169967.157797
***************************************************
Z2-amplitude:  1.00e+04
reduced chi2:  8534858087.099429
***************************************************
Z2-amplitude:  1.00e+04
reduced chi2:  8666497718.870903
***************************************************
Z2-amplitude:  1.00e+04
reduced chi2:  8877686841.0249
***************************************************
Z2-amplitude:  1.00e+04
reduced chi2:  5866032127.755403
***************************************************
Z2-amplitude:  1.00e+04
reduced chi2:  435926595.10218817
***************************************************
Z2-amplitude:  3.94e+02
reduced chi2:  7667.699630606357
***************************************************
Z2-amplitude:  4.28e+02
reduced chi2:  7761.9207376647355
***************************************************
Z2-amplitude:  9.79e+03
reduced chi2:  66901.9834298937
********************

***************************************************
Z2-amplitude:  1.54e+03
reduced chi2:  5888.995202619574
***************************************************
Z2-amplitude:  1.53e+03
reduced chi2:  6771.667471997413
***************************************************
Z2-amplitude:  1.52e+03
reduced chi2:  7957.565796601961
***************************************************
Z2-amplitude:  1.58e+03
reduced chi2:  3999.069001165491
***************************************************
Z2-amplitude:  1.56e+03
reduced chi2:  7749.5383518738045
***************************************************
Z2-amplitude:  1.52e+03
reduced chi2:  6815.433269858348
***************************************************
Z2-amplitude:  1.53e+03
reduced chi2:  5223.081869842338
***************************************************
Z2-amplitude:  1.53e+03
reduced chi2:  4394.563952584297
***************************************************
Z2-amplitude:  1.53e+03
reduced chi2:  4917.521102582212
******************

***************************************************
Z2-amplitude:  3.13e+01
reduced chi2:  5138.421551743697
***************************************************
Z2-amplitude:  1.34e+01
reduced chi2:  2987.217485337081
***************************************************
Z2-amplitude:  1.79e+00
reduced chi2:  5817.726537529166
***************************************************
Z2-amplitude:  1.43e+01
reduced chi2:  7647.915334127333
***************************************************
Z2-amplitude:  9.10e+00
reduced chi2:  5879.619480766213
***************************************************
Z2-amplitude:  9.45e+00
reduced chi2:  6813.4420476352525
***************************************************
Z2-amplitude:  9.58e+00
reduced chi2:  5451.938345974184
***************************************************
Z2-amplitude:  3.60e+01
reduced chi2:  4675.765876119452
***************************************************
Z2-amplitude:  1.61e+00
reduced chi2:  6741.402291648047
******************

In [199]:
amps = np.array([m_.values[0] for m_ in m1]) ## floating phase
phases = [m_.values[2] for m_ in m1]  ## floating phase

amps2 = np.array([m_.values[0] for m_ in m2]) ## fixed phase -2.8
phases2 = [m_.values[2] for m_ in m2]  ## fixed phase -2.8

In [200]:
_, ax = plt.subplots(1,2,figsize=(9.5,4))
ax[0].scatter(range(len(amps[10:])), amps[10:])
ax[0].scatter(range(len(amps[10:])), amps2[10:])
ax[1].scatter(range(len(amps[10:])), phases[10:])
ax[1].scatter(range(len(amps[10:])), phases2[10:])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [29]:
arrrange =  np.arange(100, 500 ,0.1)
residuals = [np.mean((amps2[10:]/scale_)%1) for scale_ in arrrange]
scale = arrrange[np.argmin(residuals)]
print(scale)

378.79999999998415


In [203]:
_,ax = plt.subplots(figsize=(9.5,4))
scale = 385

# Perrinf
ax.scatter(range(len(amps[10:])), amps[10:]/scale, label='floating-phase')
ax.scatter(range(len(amps[10:])), amps2[10:]/scale, label='fixed-phase')

ax.legend()
ax.set(xlabel='dataset', ylabel='Charges', ylim=(-10,2), xlim=(40,None))
ax.grid(axis='y')
ax.minorticks_on()
ax.grid(which='major', linestyle='-', linewidth='0.5', color='red')
ax.grid(which='minor', linestyle=':', linewidth='0.5', color='black')
ax.legend()
from matplotlib.ticker import MaxNLocator
ax.yaxis.set_major_locator(MaxNLocator(integer=True))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

### fit files

In [47]:
ll = likelihood_analyser.LikelihoodAnalyser()

def getA(i):
# for i,bdf_ in enumerate(data_bdfs2):
    print(i,'  ******************')
    fname = r'/data/new_trap/20201116/Bead2/Discharge/Neutrality_20201120/Discharge_'+str(i)+'.h5'
    bb = BeadDataFile.BeadDataFile(fname=fname)
    drive_freq = int(bb.cant_freq)
    fit_kwargs = {'A': 0, 'f': drive_freq, 'phi': phi_response, 'sigma': 5826,
                  'error_A': 1, 'error_f': 1, 'error_phi': 0.5, 'error_sigma': 10, 'errordef': 1,
                  'limit_phi': [0, 2*np.pi],  'limit_sigma': [0, 1000000],
                  'limit_A': [-1000, 1000], 
                  'print_level': 0, 'fix_f': True, 'fix_phi': True}

    m1_tmp = ll.find_mle_sin(bb.z2.copy(), drive_freq=drive_freq, fsamp=5000, bandwidth=bandwidth,
                             decimate=10, noise_rms=1, plot=False, suppress_print=True, **fit_kwargs)

    print(m1_tmp.values)
    return m1_tmp.values[0]

In [6]:
phi_response=0.3
scale = 385

In [39]:
np.arange(0,21,7)

array([ 0,  7, 14])

In [59]:
phis = np.arange(0,2*np.pi,0.2)
res_mean_all = []
res_std_all = []
for i in np.arange(0,21,0.1):
    print('***************', i)
    res_mean4 = []
    res_std4 = []
    for phi_ in phis:
        phi_response = phi_
        res_tmp = np.array(Parallel(n_jobs=15)(delayed(getA)(i) for i in tqdm(range(int(i*1000), int(i*1000)+100))))
        res_mean4.append(np.mean(res_tmp))
        res_std4.append(np.std(res_tmp))
    res_mean_all.append(res_mean4)
    res_std_all.append(res_std4)


  0%|          | 0/100 [00:00<?, ?it/s]

*************** 0.0



 63%|██████▎   | 63/100 [00:01<00:01, 22.76it/s]


  0%|          | 0/100 [00:00<?, ?it/s]

*************** 0.1


...

*************** 20.8



  0%|          | 0/100 [00:00<?, ?it/s]

*************** 20.900000000000002



100%|██████████| 100/100 [00:01<00:00, 61.88it/s]


100%|██████████| 100/100 [00:00<00:00, 225.08it/s]

In [57]:
_,ax = plt.subplots(figsize=(8,4))
ax.errorbar(phis, np.array(res_mean_all[0])/scale, yerr=np.array(res_std_all[0])/np.sqrt(100)/scale, fmt='o')
ax.errorbar(phis, np.array(res_mean_all[1])/scale, yerr=np.array(res_std_all[1])/np.sqrt(100)/scale, fmt='o')
ax.errorbar(phis, np.array(res_mean_all[2])/scale, yerr=np.array(res_std_all[2])/np.sqrt(100)/scale, fmt='o')
ax.plot(phis, phis*0, 'r--')
ax.set(xlabel='fit-phase [rad]', ylabel=r'amplitude $[\epsilon]$')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

[Text(0.5, 0, 'fit-phase [rad]'), Text(0, 0.5, 'amplitude $[\\epsilon]$')]

In [58]:
np.argmin(res_mean_all[0])

18

In [60]:
phases = np.array([phis[np.argmin(res_)] for res_ in res_mean_all])
amps = np.array([np.std(res_)*np.sqrt(2)/scale for res_ in res_mean_all])
amps0 = np.array([res_[0]/scale for res_ in res_mean_all])
std0 = np.array([res_[0]/scale/np.sqrt(100) for res_ in res_std_all])

In [61]:
_,ax = plt.subplots(2,1,figsize=(9.2,7), sharex=True)
ax[0].plot(np.arange(0,21,0.1)*1000, phases, '.--')
ax[0].set(ylabel='phase [rad]')
ax[1].plot(np.arange(0,21,0.1)*1000, amps , '.', label='amplitude')
ax[1].plot(np.arange(0,21,0.1)*1000, amps0, '.', label='response at phase=0')
ax[1].plot(np.arange(0,21,0.1)*1000, np.arange(0,21,0.1)*0, 'r--')
ax[0].set(ylabel='phase [rad]')
ax[1].set(xlabel='dataset', ylabel=r'$\epsilon$')
ax[1].legend()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [186]:
_,ax = plt.subplots(1,2,figsize=(9.2,4))
ax[0].scatter(phases, amps0)
ax[0].plot(phases, phases*0, 'r--')
ax[0].set(xlabel='phase [rad]', ylabel=r'$\epsilon$')
ax[1].hist(phases, bins=10)
ax[1].set(xlabel='phase [rad]')
print(np.mean(std0))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

0.005755975864607641


In [119]:
_,ax = plt.subplots()
ax.errorbar(phis, np.array(res_mean)/scale, yerr=np.array(res_std)/np.sqrt(1000)/scale, fmt='o')
ax.errorbar(phis, np.array(res_mean2)/scale, yerr=np.array(res_std2)/np.sqrt(1000)/scale, fmt='o')
ax.errorbar(phis, np.array(res_mean3)/scale, yerr=np.array(res_std3)/np.sqrt(1000)/scale, fmt='o')
ax.errorbar(phis, np.array(res_mean4)/scale, yerr=np.array(res_std4)/np.sqrt(1000)/scale, fmt='o')
ax.plot(phis, phis*0, 'r--')
ax.plot(2.8, np.mean(res_tmp_2_8)/scale, '*')
ax.plot(1.32, np.mean(res_tmp_1_32)/scale, '*')
ax.plot(0, np.mean(res_tmp_0)/scale, '*')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [78]:
phi0 = 0
phi_response = phi0
res_tmp_0 = np.array(Parallel(n_jobs=40)(delayed(getA)(i) for i in tqdm(range(20000))))

100%|██████████| 20000/20000 [13:14<00:00, 24.45it/s]


In [102]:
_,ax = plt.subplots()
# ax.hist(res_tmp/scale, bins=30)
# ax.hist(res_tmp_2_8/scale, bins=30)
# ax.hist(res_tmp_1_32/scale, bins=30, alpha=0.7)
ax.hist(means/scale, bins=30, alpha=0.5)
print(np.mean(res_tmp)/scale, np.std(res_tmp)/scale/np.sqrt(10000))
print(np.mean(res_tmp_2_8)/scale, np.std(res_tmp_2_8)/scale/np.sqrt(20000))
print(np.mean(res_tmp_1_32)/scale, np.std(res_tmp_1_32)/scale/np.sqrt(20000))
print(np.mean(res_tmp_0)/scale, np.std(res_tmp_0)/scale/np.sqrt(20000))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

0.0006805222121827159 0.0005603446164143654
-0.0009786831689251167 0.00040998856018796195
0.0008856047160216867 0.00041128438784253477
0.001249881834284009 0.0004087561899279354


In [85]:
tmpp = res_tmp_2_8
means = np.array([np.mean(tmpp[j*100:(j+1)*100]) for j in range(200)])
stds = np.array([np.std(tmpp[j*100:(j+1)*100]) for j in range(200)])

In [91]:
_,ax = plt.subplots(figsize=(9.5,4))
ax.errorbar(np.arange(0,20000,100), means/scale, yerr=stds/scale/np.sqrt(100),fmt='o')
ax.plot(np.arange(0,20000,100), np.arange(0,20000,100)*0, 'r--')
ax.set(xlabel='dataset', ylabel=r'$\epsilon$')
z, cov = np.polyfit(np.arange(0,20000,100), means/scale, 0, cov=True)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [95]:
z, cov, np.mean(res_mean)/scale, np.mean(res_tmp_2_8)/scale

(array([-0.00097868]),
 array([[1.94744396e-07]]),
 3.213455867827143e-05,
 -0.0009786831689251167)

### Profile Likelihood

In [300]:
index = np.random.choice(20000, 20000, replace=False)

In [301]:
def func_filter(i):
    ii = index[i]
    fname = r'/data/new_trap/20201116/Bead2/Discharge/Neutrality_20201120/Discharge_'+str(ii)+'.h5'
    bb = BeadDataFile.BeadDataFile(fname=fname)
    
    res = bb.z2
    return (res)

In [303]:
test_phases = np.array(Parallel(n_jobs=10)(delayed(func_filter)(i) for i in tqdm(range(20000))))





  0%|          | 0/20000 [00:00<?, ?it/s]



  0%|          | 10/20000 [00:01<35:51,  9.29it/s]



  0%|          | 20/20000 [00:01<32:28, 10.25it/s]



  0%|          | 30/20000 [00:02<28:41, 11.60it/s]



  0%|          | 40/20000 [00:02<25:02, 13.29it/s]



  0%|          | 50/20000 [00:03<22:46, 14.59it/s]



  0%|          | 60/20000 [00:03<20:05, 16.54it/s]


 28%|██▊       | 2770/10000 [02:36<05:24, 22.30it/s]



  0%|          | 70/20000 [00:04<19:52, 16.71it/s]



  0%|          | 80/20000 [00:04<17:08, 19.36it/s]



  0%|          | 90/20000 [00:05<16:14, 20.42it/s]



  0%|          | 100/20000 [00:05<16:12, 20.47it/s]



  1%|          | 110/20000 [00:06<16:19, 20.31it/s]



  1%|          | 120/20000 [00:06<16:43, 19.82it/s]



  1%|          | 130/20000 [00:07<16:20, 20.27it/s]



  1%|          | 140/20000 [00:07<16:03, 20.60it/s]



  1%|          | 150/20000 [00:08<16:02, 20.63it/s]



  1%|          | 160/20000 [00:08<16:30, 20.04it/s]



  1%|          | 170/2000

 12%|█▏        | 2410/20000 [01:56<15:47, 18.56it/s]



 12%|█▏        | 2420/20000 [01:56<14:29, 20.22it/s]



 12%|█▏        | 2430/20000 [01:57<14:18, 20.46it/s]



 12%|█▏        | 2440/20000 [01:57<15:30, 18.86it/s]



 12%|█▏        | 2450/20000 [01:58<14:36, 20.02it/s]



 12%|█▏        | 2460/20000 [01:58<14:17, 20.46it/s]



 12%|█▏        | 2470/20000 [01:59<13:36, 21.46it/s]



 12%|█▏        | 2480/20000 [01:59<13:59, 20.87it/s]



 12%|█▏        | 2490/20000 [02:00<14:15, 20.47it/s]



 12%|█▎        | 2500/20000 [02:00<14:45, 19.77it/s]



 13%|█▎        | 2510/20000 [02:01<14:36, 19.95it/s]



 13%|█▎        | 2520/20000 [02:01<15:39, 18.61it/s]



 13%|█▎        | 2530/20000 [02:02<15:28, 18.82it/s]



 13%|█▎        | 2540/20000 [02:02<15:45, 18.47it/s]



 13%|█▎        | 2550/20000 [02:03<15:58, 18.20it/s]



 13%|█▎        | 2560/20000 [02:03<13:44, 21.15it/s]



 13%|█▎        | 2570/20000 [02:04<13:32, 21.44it/s]



 13%|█▎        | 2580/20000 [02:04<13:47, 21.04i

 24%|██▍       | 4810/20000 [04:02<13:41, 18.49it/s]



 24%|██▍       | 4820/20000 [04:02<13:13, 19.13it/s]



 24%|██▍       | 4830/20000 [04:03<15:02, 16.81it/s]



 24%|██▍       | 4840/20000 [04:03<13:55, 18.13it/s]



 24%|██▍       | 4850/20000 [04:04<14:09, 17.83it/s]



 24%|██▍       | 4860/20000 [04:04<13:26, 18.77it/s]



 24%|██▍       | 4870/20000 [04:05<13:45, 18.34it/s]



 24%|██▍       | 4880/20000 [04:06<14:55, 16.89it/s]



 24%|██▍       | 4890/20000 [04:06<15:28, 16.27it/s]



 24%|██▍       | 4900/20000 [04:07<15:02, 16.74it/s]



 25%|██▍       | 4910/20000 [04:08<15:10, 16.58it/s]



 25%|██▍       | 4920/20000 [04:08<13:35, 18.50it/s]



 25%|██▍       | 4930/20000 [04:08<13:40, 18.37it/s]



 25%|██▍       | 4940/20000 [04:09<13:17, 18.88it/s]



 25%|██▍       | 4950/20000 [04:10<13:33, 18.51it/s]



 25%|██▍       | 4960/20000 [04:10<13:11, 18.99it/s]



 25%|██▍       | 4970/20000 [04:10<12:46, 19.61it/s]



 25%|██▍       | 4980/20000 [04:11<14:16, 17.54i

 36%|███▌      | 7210/20000 [06:12<11:21, 18.76it/s]



 36%|███▌      | 7220/20000 [06:13<12:06, 17.59it/s]



 36%|███▌      | 7230/20000 [06:13<11:32, 18.43it/s]



 36%|███▌      | 7240/20000 [06:14<12:05, 17.59it/s]



 36%|███▋      | 7250/20000 [06:14<11:11, 19.00it/s]



 36%|███▋      | 7260/20000 [06:15<11:04, 19.16it/s]



 36%|███▋      | 7270/20000 [06:15<11:21, 18.69it/s]



 36%|███▋      | 7280/20000 [06:16<10:58, 19.33it/s]



 36%|███▋      | 7290/20000 [06:16<11:10, 18.96it/s]



 36%|███▋      | 7300/20000 [06:17<11:43, 18.05it/s]



 37%|███▋      | 7310/20000 [06:17<11:38, 18.16it/s]



 37%|███▋      | 7320/20000 [06:18<12:29, 16.91it/s]



 37%|███▋      | 7330/20000 [06:19<11:49, 17.85it/s]



 37%|███▋      | 7340/20000 [06:19<11:12, 18.82it/s]



 37%|███▋      | 7350/20000 [06:20<11:38, 18.11it/s]



 37%|███▋      | 7360/20000 [06:20<12:16, 17.16it/s]



 37%|███▋      | 7370/20000 [06:21<11:13, 18.76it/s]



 37%|███▋      | 7380/20000 [06:21<11:22, 18.49i

 48%|████▊     | 9610/20000 [08:24<09:25, 18.38it/s]



 48%|████▊     | 9620/20000 [08:24<09:50, 17.58it/s]



 48%|████▊     | 9630/20000 [08:25<10:26, 16.56it/s]



 48%|████▊     | 9640/20000 [08:26<09:14, 18.70it/s]



 48%|████▊     | 9650/20000 [08:26<09:03, 19.05it/s]



 48%|████▊     | 9660/20000 [08:27<10:02, 17.18it/s]



 48%|████▊     | 9670/20000 [08:27<10:05, 17.06it/s]



 48%|████▊     | 9680/20000 [08:28<09:51, 17.46it/s]



 48%|████▊     | 9690/20000 [08:28<09:49, 17.50it/s]



 48%|████▊     | 9700/20000 [08:29<09:19, 18.41it/s]



 49%|████▊     | 9710/20000 [08:29<09:15, 18.54it/s]



 49%|████▊     | 9720/20000 [08:30<09:02, 18.96it/s]



 49%|████▊     | 9730/20000 [08:30<08:35, 19.92it/s]



 49%|████▊     | 9740/20000 [08:31<09:25, 18.14it/s]



 49%|████▉     | 9750/20000 [08:32<09:02, 18.91it/s]



 49%|████▉     | 9760/20000 [08:32<09:29, 17.99it/s]



 49%|████▉     | 9770/20000 [08:33<10:19, 16.51it/s]



 49%|████▉     | 9780/20000 [08:33<09:13, 18.47i

 60%|█████▉    | 11980/20000 [10:33<07:29, 17.83it/s]



 60%|█████▉    | 11990/20000 [10:34<07:36, 17.54it/s]



 60%|██████    | 12000/20000 [10:34<06:41, 19.94it/s]



 60%|██████    | 12010/20000 [10:35<06:59, 19.05it/s]



 60%|██████    | 12020/20000 [10:36<07:10, 18.52it/s]



 60%|██████    | 12030/20000 [10:36<07:00, 18.95it/s]



 60%|██████    | 12040/20000 [10:37<07:06, 18.67it/s]



 60%|██████    | 12050/20000 [10:37<07:22, 17.95it/s]



 60%|██████    | 12060/20000 [10:38<07:12, 18.34it/s]



 60%|██████    | 12070/20000 [10:38<07:16, 18.16it/s]



 60%|██████    | 12080/20000 [10:39<07:18, 18.07it/s]



 60%|██████    | 12090/20000 [10:39<07:05, 18.58it/s]



 60%|██████    | 12100/20000 [10:40<06:33, 20.09it/s]



 61%|██████    | 12110/20000 [10:40<06:31, 20.15it/s]



 61%|██████    | 12120/20000 [10:41<07:02, 18.63it/s]



 61%|██████    | 12130/20000 [10:41<07:21, 17.81it/s]



 61%|██████    | 12140/20000 [10:42<07:02, 18.59it/s]



 61%|██████    | 12150/20000 [1

 72%|███████▏  | 14340/20000 [12:43<05:03, 18.66it/s]



 72%|███████▏  | 14350/20000 [12:43<05:39, 16.62it/s]



 72%|███████▏  | 14360/20000 [12:44<05:29, 17.14it/s]



 72%|███████▏  | 14370/20000 [12:44<05:10, 18.15it/s]



 72%|███████▏  | 14380/20000 [12:45<05:07, 18.28it/s]



 72%|███████▏  | 14390/20000 [12:45<05:00, 18.68it/s]



 72%|███████▏  | 14400/20000 [12:46<05:16, 17.70it/s]



 72%|███████▏  | 14410/20000 [12:47<05:09, 18.07it/s]



 72%|███████▏  | 14420/20000 [12:47<05:14, 17.72it/s]



 72%|███████▏  | 14430/20000 [12:48<05:01, 18.47it/s]



 72%|███████▏  | 14440/20000 [12:48<05:03, 18.29it/s]



 72%|███████▏  | 14450/20000 [12:49<04:59, 18.53it/s]



 72%|███████▏  | 14460/20000 [12:49<05:02, 18.29it/s]



 72%|███████▏  | 14470/20000 [12:50<05:02, 18.25it/s]



 72%|███████▏  | 14480/20000 [12:50<05:00, 18.34it/s]



 72%|███████▏  | 14490/20000 [12:51<04:53, 18.74it/s]



 72%|███████▎  | 14500/20000 [12:51<04:54, 18.66it/s]



 73%|███████▎  | 14510/20000 [1

 84%|████████▎ | 16700/20000 [14:53<02:57, 18.63it/s]



 84%|████████▎ | 16710/20000 [14:53<02:55, 18.80it/s]



 84%|████████▎ | 16720/20000 [14:54<02:56, 18.58it/s]



 84%|████████▎ | 16730/20000 [14:54<03:05, 17.65it/s]



 84%|████████▎ | 16740/20000 [14:55<02:56, 18.42it/s]



 84%|████████▍ | 16750/20000 [14:55<02:54, 18.58it/s]



 84%|████████▍ | 16760/20000 [14:56<02:53, 18.68it/s]



 84%|████████▍ | 16770/20000 [14:56<02:56, 18.26it/s]



 84%|████████▍ | 16780/20000 [14:57<02:47, 19.21it/s]



 84%|████████▍ | 16790/20000 [14:58<03:00, 17.79it/s]



 84%|████████▍ | 16800/20000 [14:58<03:06, 17.17it/s]



 84%|████████▍ | 16810/20000 [14:59<02:40, 19.84it/s]



 84%|████████▍ | 16820/20000 [14:59<02:57, 17.87it/s]



 84%|████████▍ | 16830/20000 [15:00<02:55, 18.02it/s]



 84%|████████▍ | 16840/20000 [15:00<03:01, 17.40it/s]



 84%|████████▍ | 16850/20000 [15:01<03:01, 17.31it/s]



 84%|████████▍ | 16860/20000 [15:01<02:49, 18.53it/s]



 84%|████████▍ | 16870/20000 [1

 95%|█████████▌| 19060/20000 [17:03<00:51, 18.26it/s]



 95%|█████████▌| 19070/20000 [17:04<00:50, 18.36it/s]



 95%|█████████▌| 19080/20000 [17:04<00:51, 17.71it/s]



 95%|█████████▌| 19090/20000 [17:05<00:49, 18.34it/s]



 96%|█████████▌| 19100/20000 [17:05<00:48, 18.46it/s]



 96%|█████████▌| 19110/20000 [17:06<00:50, 17.53it/s]



 96%|█████████▌| 19120/20000 [17:07<00:46, 18.94it/s]



 96%|█████████▌| 19130/20000 [17:07<00:47, 18.37it/s]



 96%|█████████▌| 19140/20000 [17:08<00:46, 18.35it/s]



 96%|█████████▌| 19150/20000 [17:08<00:47, 17.86it/s]



 96%|█████████▌| 19160/20000 [17:09<00:46, 17.95it/s]



 96%|█████████▌| 19170/20000 [17:09<00:44, 18.49it/s]



 96%|█████████▌| 19180/20000 [17:10<00:43, 18.90it/s]



 96%|█████████▌| 19190/20000 [17:10<00:42, 18.92it/s]



 96%|█████████▌| 19200/20000 [17:11<00:42, 18.85it/s]



 96%|█████████▌| 19210/20000 [17:11<00:42, 18.48it/s]



 96%|█████████▌| 19220/20000 [17:12<00:41, 18.93it/s]



 96%|█████████▌| 19230/20000 [1

In [324]:
ll = likelihood_analyser.LikelihoodAnalyser()

def getPL(i):
    print(i,'  ******************')
#     fname = r'/data/new_trap/20201116/Bead2/Discharge/Neutrality_20201120/Discharge_'+str(i)+'.h5'
#     bb = BeadDataFile.BeadDataFile(fname=fname)
    drive_freq = 71
    data_z2 = np.mean(test_phases[i*1000:(i+1)*1000], axis=0)

    fit_kwargs = {'A': 0, 'f': drive_freq, 'phi': phi_response, 'sigma': 5826,
                  'error_A': 1, 'error_f': 1, 'error_phi': 0.5, 'error_sigma': 10, 'errordef': 1,
                  'limit_phi': [0, np.pi],  'limit_sigma': [0, 1000000],
                  'limit_A': [-1000, 1000], 
                  'print_level': 0, 'fix_f': True, 'fix_phi': True}

    m1_tmp = ll.find_mle_sin(data_z2, drive_freq=drive_freq, fsamp=5000, bandwidth=bandwidth,
                             decimate=10, noise_rms=1, plot=False, suppress_print=True, **fit_kwargs)

    print('MLEs: ', m1_tmp.values)
    A_array = np.arange(-10,10,0.1)
    PL_array = 2*10*ll.get_PL_sin(A_array=A_array, **fit_kwargs)
    PL_array -= np.min(np.interp(np.arange(-10,10,0.001), A_array, PL_array))
    
    return PL_array

In [325]:
PL_test2 = Parallel(n_jobs=40)(delayed(getPL)(i) for i in tqdm(range(20)))





  0%|          | 0/20 [00:00<?, ?it/s]



100%|██████████| 20/20 [00:00<00:00, 9082.51it/s]

In [326]:
A_array = np.arange(-10,10,0.1)
PLs_ = PL_test2
PLs5_tmp2 = PLs_[0].copy()
for pl in PLs_[1:]:
    PLs5_tmp2 += pl
PLs5_tmp2 -= np.min(np.interp(np.arange(-10,10,0.001), A_array, PLs5_tmp2))

In [330]:
A_array = np.arange(-10,10,0.1)
PLs_tmp5 = []
for pl in PL_test2.copy():
    min_alpha = A_array[np.argmin(pl)]
    if (min_alpha<0.001):
        for i,A_ in enumerate(A_array):
            if A_>min_alpha:
                pl[i] = 0
        PLs_tmp5.append(pl)
    else:
        PLs_tmp5.append(A_array*0)

In [331]:
A_array = np.arange(-10,10,0.1)
PLs_ = PLs_tmp5
PLs5_20k_rand2 = PLs_[0].copy()
for pl in PLs_[1:]:
    PLs5_20k_rand2 += pl
PLs5_20k_rand2 -= np.min(np.interp(np.arange(-10,10,0.001), A_array, PLs5_20k_rand2))

In [333]:
A_array = np.arange(-10,10,0.1)/scale
_,ax = plt.subplots()
# ax.plot(A_array, PLs5_tmp, '*--')
# ax.plot(A_array, PLs5_tmp2, '*--')
ax.plot(A_array, PLs5_20k_rand, '*--')
ax.plot(A_array, PLs5_20k_rand2, '*--')
ax.plot(A_array, A_array*0 + 2.71, 'r--')
_ = ax.set(ylim=(0,6), xlim=(-0.01,0.01), xlabel=r'$\epsilon_{fit}$', ylabel = 'Profile-Likelihood')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

### Phase stability - 7.5um

In [16]:
ll = likelihood_analyser.LikelihoodAnalyser()

def getPhase(i):
    print(i,'  ******************')
    fname = r'/data/new_trap/20201203/Bead2/Discharge/SeismometerTest/Discharge_'+str(i)+'.h5'
    bb = BeadDataFile.BeadDataFile(fname=fname)
    drive_freq = int(bb.cant_freq)
    fit_kwargs = {'A': 0, 'f': drive_freq, 'phi': phi_response, 'sigma': 5826,
                  'error_A': 1, 'error_f': 1, 'error_phi': 0.5, 'error_sigma': 10, 'errordef': 1,
                  'limit_phi': [-np.pi, np.pi],  'limit_sigma': [0, 1000000],
                  'limit_A': [-1000, 0], 
                  'print_level': 0, 'fix_f': True, 'fix_phi': False}

    m1_tmp = ll.find_mle_sin(bb.z2.copy(), drive_freq=drive_freq, fsamp=5000, bandwidth=bandwidth,
                             decimate=10, noise_rms=1, plot=False, suppress_print=True, **fit_kwargs)

    print(m1_tmp.values)
    return m1_tmp.values[2]

In [11]:
phi_response=0.3
scale = 385
bandwidth = 2

In [18]:
phase_response2 = Parallel(n_jobs=40)(delayed(getPhase)(i) for i in tqdm(range(10, 18000)))


 33%|███▎      | 5920/17990 [04:20<09:10, 21.92it/s]


 65%|██████▌   | 11720/17990 [08:41<04:40, 22.34it/s]


 97%|█████████▋| 17440/17990 [13:01<00:24, 22.81it/s]


100%|██████████| 17990/17990 [13:24<00:00, 22.37it/s]

In [28]:
_,ax = plt.subplots(2,1,figsize=(9.2,5.5), sharex=True)
ax[0].scatter(np.arange(len(phase_response)), phase_response2)
ax[0].set(ylim=(-np.pi, np.pi), ylabel='phase response[rad]')

ax[1].scatter(np.arange(len(phase_response)), phase_response)
ax[1].set(xlabel='datafile', ylabel='amp response[a.u.]')

print(np.mean(phase_response))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

-317.5687623294259


In [21]:
_,ax = plt.subplots(figsize=(9.2,3.5))
ax.scatter(np.arange(len(phase_response)), phase_response)
ax.set(xlabel='datafile', ylabel='amp response[a.u.]')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

[Text(0.5, 0, 'datafile'), Text(0, 0.5, 'amp response[a.u.]')]

In [36]:
amp_filt = [a for a,b in zip(phase_response,phase_response2) if (a>-420)&(a<-250)]
phase_filt = [b for a,b in zip(phase_response,phase_response2) if (a>-420)&(a<-250)]

In [38]:
_,ax = plt.subplots()
ax.scatter(phase_filt, amp_filt)
ax.set(xlabel='phase response[rad]', ylabel='amp response[a.u.]')
print(np.std(phase_filt))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

0.1288075906018014
